In [ ]:
from optimed.processes.preview import preview_3d_image
from optimed.wrappers.operations import join, exists, isfile, maybe_mkdir
from tqdm import tqdm
import random
import os
from multiprocessing import Pool, cpu_count

dst = "/home/eolika/Desktop/BlueMind/nnUNet/trains/nnUNet_raw/Dataset101_mosaic_skelet_spine"
imgs_dir = join(dst, "imagesTr")
labels_dir = join(dst, "labelsTr")

cases = sorted([i for i in os.listdir(labels_dir) if isfile(join(labels_dir, i))])

segmentation_dict = {
    label: {
        'color': (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),
        'opacity': 1.0
    }
    for label in range(1, 27)
}

preview_dir = join(dst, "preview")
maybe_mkdir(preview_dir)

def process_case(case):
    base = case.split('.nii.gz')[0]
    # Проверяем, существует ли уже изображение.
    # Обратите внимание, что в оригинале условие проверяет путь join(dst, case, f"{base}.png")
    # Если требуется проверять именно preview_dir, можно заменить путь ниже.
    if not exists(join(preview_dir, f"{base}.png")):
        preview_3d_image(
            input_path=join(labels_dir, case),
            output=join(preview_dir, f"{base}.png"),
            segmentation_dict=segmentation_dict,
            view_direction=["A", "P", "R", "L"],
            background_color=(255, 255, 255),
            window_size=(1200, 1200)
        )
    return case

# processes=cpu_count()
processes=4

if __name__ == "__main__":
    # Создаём пул процессов, используя количество ядер CPU
    with Pool(processes=processes) as pool:
        # Используем imap для получения результатов и обновления progress bar
        for _ in tqdm(pool.imap_unordered(process_case, cases), total=len(cases), desc="Processing cases"):
            pass
